In [ ]:
#| default_exp models

In [ ]:
from looped_experiments.nano_gpt import GPT, GPTConfig

In [ ]:
class LoopedTransformer:
    def __init__(self, config):
        cfg = GPTConfig()
        self.backbone = GPT(cfg)